In [ ]:
%load_ext tensorboard

from datetime import datetime

import tensorflow as tf
import numpy as np      

#from tqdm import tqdm

In [ ]:
import sys, os
from pathlib import Path

In [ ]:
sys.path.append('../')

In [ ]:
from adinilm.utils import paths_manager as pathsman

In [ ]:
selected_profile = pathsman.PROFILES_DIR / "unetnilm_ukdale_20240321_155419"

print(f"Selected profile {selected_profile.resolve()} {'exists' if selected_profile.exists() else 'does not exist'}")

In [ ]:
denoise_inputs = np.load(selected_profile / "training" / "denoise_inputs.npy")
noise_inputs = np.load(selected_profile / "training" / "noise_inputs.npy")
targets = np.load(selected_profile / "training" / "targets.npy")
states = np.load(selected_profile / "training" / "states.npy")

In [ ]:
def split_data(data):
	split_1 = int(0.60 * len(data))
	split_2 = int(0.85 * len(data))
	train = data[:split_1]
	validation = data[split_1:split_2]
	test = data[split_2:]
	return train, validation, test


In [ ]:
class CustomDataLoader(tf.keras.utils.Sequence):

        class_dict = {"fridge" : 0, "washer dryer" : 1, "kettle" : 2, "dish washer" : 3, "microwave" : 4}

        def __init__(self, profile_path, d_type, batch_size=256, seq_len=100, denoise=True, *args, **kwargs):
                super(CustomDataLoader, self).__init__(*args, **kwargs)

                self.profile_path = Path(profile_path)
                self.batch_size = batch_size
                self.seq_len = seq_len
                self.d_type = d_type 
                self.denoise = denoise
                self.data, self.labels = self.__load()
                self.indices = np.arange(self.data.shape[0])

        def __load(self):

                if self.denoise:
                        x = np.load(self.profile_path / "training" / "denoise_inputs.npy")
                else:
                        x = np.load(self.profile_path / "training" / "noise_inputs.npy")
                y = np.load(self.profile_path / "training" / "targets.npy")
                z = np.load(self.profile_path / "training" / "states.npy")

                train_x, val_x, test_x = split_data(x)
                train_y, val_y, test_y = split_data(y)
                train_z, val_z, test_z = split_data(z)          

                if self.d_type == "train":
                        x = train_x
                        y = train_y
                        z = train_z
                elif self.d_type == "test":
                        x = test_x
                        y = test_y
                        z = test_z
                else:
                        x = val_x
                        y = val_y
                        z = val_z

                return x, (z, y)

        def __len__(self):
                return (self.data.shape[0] - self.seq_len) // self.batch_size

        def get_sample(self, index):
                indices = self.indices[index : index + self.seq_len]
                inds_inputs = sorted(indices[:self.seq_len])
                inds_labels = sorted(indices[self.seq_len-1:self.seq_len])
		
                states = self.labels[0]
                power = self.labels[1]
        
                return self.data[inds_inputs], (states[inds_labels], power[inds_labels])

        def __getitem__(self, index):
                batch = [self.get_sample(idx) for idx in range(index * self.batch_size,(index + 1) * self.batch_size)]
                batch_data, batch_label = zip(*batch)
                
                batch_data = np.array(batch_data)
                batch_label = np.array(batch_label)

                batch_label = tf.squeeze(batch_label, axis=2)

                return batch_data, tuple([batch_label[:,0,:], batch_label[:,1,:]])

In [ ]:
seq_len = 100
denoise = True
dloader = CustomDataLoader(selected_profile, "train", 256, seq_len=seq_len)


In [ ]:
def create_model(input_window_length):

    """Specifies the structure of a seq2point model using Keras' functional API.

    Returns:
    model (tensorflow.keras.Model): The uncompiled seq2point model.

    """
    input_layer = tf.keras.layers.Input(shape=(input_window_length,))
    reshape_layer = tf.keras.layers.Reshape((1, input_window_length, 1))(input_layer)
    conv_layer_1 = tf.keras.layers.Convolution2D(filters=30, kernel_size=(10, 1), strides=(1, 1), padding="same", activation="relu")(reshape_layer)
    conv_layer_2 = tf.keras.layers.Convolution2D(filters=30, kernel_size=(8, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_1)
    conv_layer_3 = tf.keras.layers.Convolution2D(filters=40, kernel_size=(6, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_2)
    conv_layer_4 = tf.keras.layers.Convolution2D(filters=50, kernel_size=(5, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_3)
    conv_layer_5 = tf.keras.layers.Convolution2D(filters=50, kernel_size=(5, 1), strides=(1, 1), padding="same", activation="relu")(conv_layer_4)
    flatten_layer = tf.keras.layers.Flatten()(conv_layer_5)
    label_layer = tf.keras.layers.Dense(1024, activation="relu")(flatten_layer)
    output_layer_1 = tf.keras.layers.Dense(2*5, activation="linear", name="y1_output")(label_layer)
    output_layer_2 = tf.keras.layers.Dense(5*5, activation="linear", name="y2_output")(label_layer)

    model = tf.keras.Model(inputs=input_layer, outputs=[output_layer_1, output_layer_2])
    return model

In [ ]:
def create_resnet_model(input_window_length):
    
        input_layer = tf.keras.layers.Input(shape=(input_window_length,))
        reshape_layer = tf.keras.layers.Reshape((1, input_window_length, 1))(input_layer)
        
        conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv1')(reshape_layer)
        bn1 = tf.keras.layers.BatchNormalization(name='bn1')(conv1)
        conv2 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv2')(bn1)
        bn2 = tf.keras.layers.BatchNormalization()(conv2)
        conv3 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='bn2')(bn2)
        resid_1 = tf.keras.layers.Add()([bn2, conv3])
        relu1 = tf.keras.layers.ReLU(name='relu1')(resid_1)
        bn_res1 = tf.keras.layers.BatchNormalization(name='bn_res1')(relu1)

        conv4 = tf.keras.layers.Conv2D(filters=48, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv4')(bn_res1)
        bn4 = tf.keras.layers.BatchNormalization(name='bn4')(conv4)
        conv5 = tf.keras.layers.Conv2D(filters=48, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv5')(bn4)
        bn5 = tf.keras.layers.BatchNormalization(name='bn5')(conv5)
        pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name="pool2")(bn5)
        conv6 = tf.keras.layers.Conv2D(filters=48, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv6')(pool2)
        resid_2 = tf.keras.layers.Add()([pool2, conv6])
        relu2 = tf.keras.layers.ReLU(name="relu2")(resid_2)
        bn_res2 = tf.keras.layers.BatchNormalization(name='bn_res2')(relu2)

        conv7 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv7')(bn_res2)
        bn7 = tf.keras.layers.BatchNormalization(name='bn7')(conv7)
        conv8 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv8')(bn7)
        bn8 = tf.keras.layers.BatchNormalization(name='bn8')(conv8)
        pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name='pool3')(bn8)
        conv9 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name='conv9')(pool3)
        resid_3 = tf.keras.layers.Add()([pool3, conv9])
        relu3 = tf.keras.layers.ReLU(name="relu3")(resid_3)
        bn_res3 = tf.keras.layers.BatchNormalization(name="bn_res3")(relu3)

        conv10 = tf.keras.layers.Conv2D(filters=96, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name="conv10")(bn_res3)
        bn10 = tf.keras.layers.BatchNormalization(name="bn10")(conv10)
        conv11 = tf.keras.layers.Conv2D(filters=96, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name="conv11")(bn10)
        bn11 = tf.keras.layers.BatchNormalization(name="bn11")(conv11)
        pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name="pool4")(bn11)
        conv12 = tf.keras.layers.Conv2D(filters=96, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name="conv12")(pool4)
        resid_4 = tf.keras.layers.Add()([pool4, conv12])
        relu4 = tf.keras.layers.ReLU(name="relu4")(resid_4)
        bn_res4 = tf.keras.layers.BatchNormalization(name="bn_res4")(relu4)

        conv13 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name="conv13")(bn_res4)
        pool5 = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name="pool5")(conv13)
        bn13 = tf.keras.layers.BatchNormalization(name="bn13")(pool5)

        conv14 = tf.keras.layers.Conv2D(filters=96, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name="conv14")(bn13)
        bn14 = tf.keras.layers.BatchNormalization(name="bn14")(conv14)

        conv15 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 1), strides=(1, 1), padding="same", activation="relu", name="conv15")(bn14)
        bn15 = tf.keras.layers.BatchNormalization(name="bn15")(conv15)

        flatten_layer = tf.keras.layers.Flatten()(bn15)
        
        label_layer = tf.keras.layers.Dense(256, activation="relu", name="dense")(flatten_layer)
        output_layer_1 = tf.keras.layers.Dense(2*5, activation="linear", name="y1_output")(label_layer)
        output_layer_2 = tf.keras.layers.Dense(5*5, activation="linear", name="y2_output")(label_layer)

        model = tf.keras.Model(inputs=input_layer, outputs=[output_layer_1, output_layer_2])
        return model



In [ ]:
# mod = create_model(input_window_length=seq_len)
mod = create_resnet_model(input_window_length=seq_len)

dot_img_file = '/tmp/resnet_model.png'
tf.keras.utils.plot_model(mod, to_file=dot_img_file, show_shapes=True)

mod.summary()

In [ ]:
input_window_length = 100

In [ ]:
class MultiActivationLoss(tf.keras.Loss):
        def __init__(self, *args, **kwargs):
                super().__init__(*args, **kwargs)

                self.states_loss = tf.keras.losses.SparseCategoricalCrossentropy()

        def call(self, y_true, y_pred):

                pred_state = tf.reshape(y_pred, [-1, 5, 2])

                pred_state_softmax = tf.nn.softmax(pred_state, axis=1)

                loss_nll = self.states_loss(y_true, pred_state_softmax)

                return loss_nll

In [ ]:
class QuantileLoss(tf.keras.Loss):
        def __init__(self, quantiles=[0.0025,0.1, 0.5, 0.9, 0.975], *args, **kwargs):
                super().__init__(*args, **kwargs)
                self.quantiles = quantiles

        def call(self, y_true, y_pred):
                pred_rms = tf.reshape(y_pred, [-1, 5, 5])

                targets = tf.expand_dims(y_true, axis=1)
                targets = tf.repeat(targets, repeats=[5], axis=1)

                quantiles = tf.convert_to_tensor(self.quantiles, dtype=tf.float32)
                
                error = tf.transpose((targets - pred_rms), perm=[0,2,1])
                loss = tf.math.maximum(quantiles*error, (quantiles-1)*error)
                return tf.reduce_mean(loss, axis=1)

In [ ]:
mod.compile(optimizer=tf.keras.optimizers.Adam(),
              loss={'y1_output' : MultiActivationLoss, 'y2_output' : QuantileLoss})

In [23]:
# Define the Keras TensorBoard callback.
# Refer to https://stackoverflow.com/questions/56690089/how-to-graph-tf-keras-model-in-tensorflow-2-0

logdir="../logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
checkpoint_path = f"{logdir}/weights/cp.weights.h5" 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

mod.fit(dloader, epochs=1, callbacks=[tensorboard_callback, checkpoint_callback])
%tensorboard --logdir logs

2134/2577 ━━━━━━━━━━━━━━━━━━━━ 1:21 184ms/step - loss: 0.4958